<a href="https://colab.research.google.com/github/juangrave9/DataViz/blob/main/Data_visualization_%3D%3E_Pandas_%26_Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generación de visualizaciones de data a partir de un Dataset

In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import csv

path='https://raw.githubusercontent.com/juangrave9/DataViz/95e19e4724cfec38d26b9fba70e4ab90e5fe3615/dataset_incendios.csv'

In [2]:
df = pd.read_csv(path,sep=';')

In [3]:
df

,pais_id,pais,provincia_id,provincia,departamento_id,departamento,sup_afectada,uni_med_id,cant_focos,año_inicial,año_final
0,32,Argentina,6,Buenos Aires,833,Tres Arroyos,2400.00,ha,1,2014,2014
1,32,Argentina,14,Córdoba,14,Calamuchita,50.00,ha,1,2015,2015
2,32,Argentina,18,Corrientes,56,General Alvear,257.00,ha,1,2012,2012
3,32,Argentina,18,Corrientes,28,Concepción,130.00,ha,2,2012,2012
4,32,Argentina,18,Corrientes,28,Concepción,5.00,ha,1,2013,2013
...,...,...,...,...,...,...,...,...,...,...,...
115,32,Argentina,26,Chubut,98,Tehuelches,0.30,ha,1,2014,2015
116,32,Argentina,26,Chubut,98,Tehuelches,0.20,ha,1,2015,2016
117,32,Argentina,58,Neuquén,84,Ñorquin,240.80,ha,2,2011,2012
118,32,Argentina,58,Neuquén,84,Ñorquin,13.49,ha,2,2012,2013


In [4]:
print('Dimension de la data sin filtrado por nulos',df.shape)
df.isnull().sum()

Dimension de la data sin filtrado por nulos (120, 11)


pais_id            0
pais               0
provincia_id       0
provincia          0
departamento_id    0
departamento       0
sup_afectada       2
uni_med_id         0
cant_focos         0
año_inicial        0
año_final          0
dtype: int64

La baja cantidad de nulos, sobre una variable de interés importante, lleva a dropear los nulos en el DF (2 instancias en 120)

In [5]:
df.dropna(inplace=True)
print(df.shape)

(118, 11)


Se realiza un agrupamiento por provincia para la visualización, calculando dos series de interés analítico:
* Cantidad de focos de incendio por provincia
* Cantidad de hectareas afectadas por provincia

In [6]:
df['sup_afectada'].apply(lambda x:float(x)) #casteo a float de los valores dados
serie_hect=df.groupby(['provincia'])['sup_afectada'].agg('sum') #
serie_focos=df.groupby(['provincia'])['cant_focos'].agg('sum')

In [7]:
#Concatenamos en un DF las series generadas para mayor comodidad de manejo en la estructura
prov_df=pd.concat((serie_hect,serie_focos),axis=1)

In [8]:
prov_df.head()

,sup_afectada,cant_focos
provincia,,
Buenos Aires,2400.00,1
Chubut,513.01,126
Corrientes,5439.55,55
Córdoba,50.00,1
Entre Ríos,594.00,12


## Gráfico de torta (pie chart) : visualización de hectáreas afectadas agrupadas por provincia

In [9]:
r1=min(df['año_inicial'])
r2=max(df['año_final']) #Encontramos los límites para la generación del piechart

px.pie(prov_df,prov_df.index,'sup_afectada', title=f'Proporción de hectáreas afectadas por incendios en Argentina - Periodo {r1}-{r2} ',) 

##Grafico de barras (barchart) con uso de colorscale evaluando hectáreas afectadas

In [10]:
prov_df.sort_values(by='sup_afectada',inplace=True)
texts=[]
for i in range(0,prov_df.shape[0]):
  if prov_df.iloc[i,1]==1:
    texts.append( ( str(prov_df.iloc[i,1]) +' foco' ))
  else:
    texts.append( ( str(prov_df.iloc[i,1]) +' focos' ))


In [11]:
texts

['2 focos',
 '2 focos',
 '3 focos',
 '1 foco',
 '543 focos',
 '126 focos',
 '12 focos',
 '86 focos',
 '10 focos',
 '80 focos',
 '1 foco',
 '62 focos',
 '55 focos']

In [12]:
serie_focos

provincia
Buenos Aires      1
Chubut          126
Corrientes       55
Córdoba           1
Entre Ríos       12
Jujuy            62
La Pampa          3
Mendoza          80
Misiones         10
Neuquén          86
Río Negro       543
San Juan          2
Tucuman           2
Name: cant_focos, dtype: int64

In [13]:


fig = px.bar(prov_df, x=prov_df.index, y='sup_afectada', 
       title=f'Superficie afectada y cantidad de focos de incendio en Argentina - Periodo {r1}-{r2} ',
       log_y=True,orientation='v',text=texts, hover_name='cant_focos'       ,
       color='sup_afectada', color_continuous_scale='cividis' ,
       width=35*36, height=18*36        ) 

fig.update_layout(coloraxis_colorbar=dict(title="Superficie afectada [ha]" ) )

fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="sans-serif bold",
    title_font_color="black")

fig.update_xaxes(title_font_family="Arial",title='Provincias')
fig.update_yaxes(title_font_family="Arial",title='Superficie Afectada [ha]')

fig.show()



##Gráfico de barras con selección de los n departamentos más afectados (desde linea de código)

In [14]:
limitvalue=10

limit_df = df.sort_values(by='sup_afectada',ascending=False).iloc[0:limitvalue,:]
fig = px.bar(limit_df, x='departamento', y='sup_afectada', 
       title=f'Departamentos más afectados por incendios en el país. Periodo {r1}-{r2}',text='provincia',
       orientation='v', hover_name='cant_focos'       ,
       color='sup_afectada', color_continuous_scale='cividis' ,
       width=35*36, height=18*36        ) 

fig.update_layout(coloraxis_colorbar=dict(title="Superficie afectada [ha]" ) )

fig.update_layout(
    font_family="Arial",
    font_color="black",
    title_font_family="sans-serif bold",
    title_font_color="black")

fig.update_xaxes(title_font_family="Arial",title='Departamentos')
fig.update_yaxes(title_font_family="Arial",title='Superficie afectada [ha]')

fig.show()

